In [1]:
# Import environment loading library
from dotenv import load_dotenv
import json
# Import IBMGen Library 
from langchain.chains.conversation.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from langchain.llms.base import LLM
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
# Import lang Chain Interface object
# from langChainInterface import LangChainInterface
# Import system libraries
import os
# Import streamlit for the UI 
import streamlit as st
import re
from ibm_watson_machine_learning.foundation_models import Model
import docx
import pandas as pd

# Load environment vars
load_dotenv()

# Define credentials 
api_key = os.getenv("API_KEY", None)
ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None)
project_id = os.getenv("PROJECT_ID", None)
serp_api_key = os.getenv("SERP_API_KEY",None)


if api_key is None or ibm_cloud_url is None or project_id is None:
    print("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }


Forcasting Revenue 

In [2]:
df_all = pd.read_csv("data_sum_per_region_date_all_dept.csv")
df_all.head(75)

,Department,YearMonth,Revenue,Revenue Goal,Margin,Margin Goal,Sales Quantity,Customers
0,Account Management,2017-01,101963.34,96135.54,5.55,5.33,800,2911
1,Account Management,2017-02,101472.90,94475.08,5.00,4.78,793,3208
2,Account Management,2017-03,102730.67,100939.24,4.84,5.23,755,2513
3,Account Management,2017-04,113780.54,96815.29,5.19,5.24,864,3496
4,Account Management,2017-05,122237.61,96566.76,5.49,5.34,965,3435
...,...,...,...,...,...,...,...,...
70,Account Management,2022-11,64799.64,51287.27,5.65,5.21,898,5187
71,Account Management,2022-12,58283.11,46283.89,5.66,5.52,842,4987
72,Business Development,2017-01,106506.71,107537.37,5.51,5.40,822,3055
73,Business Development,2017-02,90945.77,105444.45,4.57,4.91,693,2633


In [3]:
df_new = df_all[["Department", "Revenue", "YearMonth"]]

df_new

,Department,Revenue,YearMonth
0,Account Management,101963.34,2017-01
1,Account Management,101472.90,2017-02
2,Account Management,102730.67,2017-03
3,Account Management,113780.54,2017-04
4,Account Management,122237.61,2017-05
...,...,...,...
1363,Strategic Partnerships,75280.40,2022-08
1364,Strategic Partnerships,63078.50,2022-09
1365,Strategic Partnerships,78387.13,2022-10
1366,Strategic Partnerships,82990.05,2022-11


In [4]:
df_new["YearMonth"] = pd.to_datetime(df_new["YearMonth"])
df_2022_11 = df_new[df_new["YearMonth"] <= "2022-11"]
df_2022_11[["Department", "Revenue", "YearMonth"]].tail()



C:\Users\MRaihan\AppData\Local\Temp\ipykernel_8184\767998096.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["YearMonth"] = pd.to_datetime(df_new["YearMonth"])


,Department,Revenue,YearMonth
1362,Strategic Partnerships,54518.15,2022-07-01
1363,Strategic Partnerships,75280.40,2022-08-01
1364,Strategic Partnerships,63078.50,2022-09-01
1365,Strategic Partnerships,78387.13,2022-10-01
1366,Strategic Partnerships,82990.05,2022-11-01


In [5]:
def revenue_generator(df,i, creds = creds, project_id = project_id):
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 100,
            "min_new_tokens": 1,
            "repetition_penalty": 1,
            "stop_sequences": ["\n\n"]
        }
        llama2 = "meta-llama/llama-2-70b-chat"
        model = Model(
                model_id=llama2,
                params=parameters,
                credentials=creds,
                project_id=project_id)
        
        template = "<[INST] «SYS»\n"\
                "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible\n"\
                "while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\n"\
                "Please ensure that your responses are socially unbiased and positive in nature.\n"\
                "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n"\
                "If you don't know the answer to a question, please don't share false information.\n"\
                "«/SYS»\n"\
                f"{df} is a dataset of a company.\n"\
                "Predict the next one month revenue based on the data provided.\n"\
                "Please give the answear only the value of the prediction.\n"\
                "[/INST]\n"\
                "Output:"
                

        # Generate text using the model
        generated_response = model.generate_text(prompt=template)

    # Return the results directly as JSON
        return generated_response


Prediction without crosschecking (Predicting Revenue Value 2023)

In [6]:
dep_name = df_new['Department'].unique().tolist()
df_per_team = pd.DataFrame()
result_list = []
for i in dep_name:
    print(f'Process Running for Departement{i}')
    df_per_team = df_new[df_new['Department']==i]
    result = revenue_generator(df_per_team, i)
    result_json = {i:result}
    print(result_json)
    result_list.append(result_json)
    
    print(f'Process Complete for Departement {i}')

result_list    
    

Process Running for DepartementAccount Management


{'Account Management': ' 59824.87'}
Process Complete for Departement Account Management
Process Running for DepartementBusiness Development
{'Business Development': ' 55000.00'}
Process Complete for Departement Business Development
Process Running for DepartementChannel Sales
{'Channel Sales': ' 32,000.00'}
Process Complete for Departement Channel Sales
Process Running for DepartementCustomer Success
{'Customer Success': ' 74265.89'}
Process Complete for Departement Customer Success
Process Running for DepartementE-commerce Sales
{'E-commerce Sales': ' 81212.85'}
Process Complete for Departement E-commerce Sales
Process Running for DepartementEnterprise Sales
{'Enterprise Sales': ' 72921.23'}
Process Complete for Departement Enterprise Sales
Process Running for DepartementInside Sales
{'Inside Sales': ' 93421.25\n\n'}
Process Complete for Departement Inside Sales
Process Running for DepartementInternational Sales
{'International Sales': ' 58924.89'}
Process Complete for Departement Int

[{'Account Management': ' 59824.87'},
 {'Business Development': ' 55000.00'},
 {'Channel Sales': ' 32,000.00'},
 {'Customer Success': ' 74265.89'},
 {'E-commerce Sales': ' 81212.85'},
 {'Enterprise Sales': ' 72921.23'},
 {'Inside Sales': ' 93421.25\n\n'},
 {'International Sales': ' 58924.89'},
 {'Key Accounts': ' 125000\n\n'},
 {'Outside Sales': ' 75000\n\n'},
 {'Retail Sales': ' 792  Retail Sales  102784.99 2023-01-01\n\n'},
 {'Sales Administration': ' 25000\n\n'},
 {'Sales Analytics': ' 125000\n\n'},
 {'Sales Enablement': ' 25,548.93'},
 {'Sales Engineering': ' 26878.89'},
 {'Sales Operations': ' 80543.42'},
 {'Sales Support': ' 94212.85'},
 {'Sales Training': ' 1296  Sales Training  125000.00 2023-01-01\n\n'},
 {'Strategic Partnerships': ' 88243.87\n\n'}]

Prediction With Checking (Predicting revenue value 2022)

In [7]:
dep_name = df_2022_11['Department'].unique().tolist()
df_per_team = pd.DataFrame()
result_list1 = []
for i in dep_name:
    print(f'Process Running for Departement{i}')
    df_per_team = df_2022_11[df_2022_11['Department']==i]
    result = revenue_generator(df_per_team, i)
    result_json = {i:result}
    print(result_json)
    result_list.append(result_json)
    
    print(f'Process Complete for Departement {i}')
    print("prediction has been completed.")

print(result_list1)    

Process Running for DepartementAccount Management
{'Account Management': ' 52845.89'}
Process Complete for Departement Account Management
prediction has been completed.
Process Running for DepartementBusiness Development
{'Business Development': ' 54250.89'}
Process Complete for Departement Business Development
prediction has been completed.
Process Running for DepartementChannel Sales
{'Channel Sales': ' 25,000.00'}
Process Complete for Departement Channel Sales
prediction has been completed.
Process Running for DepartementCustomer Success
{'Customer Success': ' 55642.89'}
Process Complete for Departement Customer Success
prediction has been completed.
Process Running for DepartementE-commerce Sales
{'E-commerce Sales': ' 75000\n\n'}
Process Complete for Departement E-commerce Sales
prediction has been completed.
Process Running for DepartementEnterprise Sales
{'Enterprise Sales': ' 59878.92'}
Process Complete for Departement Enterprise Sales
prediction has been completed.
Process Run

In [8]:
df_2022_12 = df_new[df_new["YearMonth"] == "2022-12"]
print(df_2022_12)

                  Department    Revenue  YearMonth
71        Account Management   58283.11 2022-12-01
143     Business Development   52286.02 2022-12-01
215            Channel Sales   30134.25 2022-12-01
287         Customer Success   71357.59 2022-12-01
359         E-commerce Sales  104492.57 2022-12-01
431         Enterprise Sales   69478.81 2022-12-01
503             Inside Sales   91318.56 2022-12-01
575      International Sales   63081.57 2022-12-01
647             Key Accounts  113740.07 2022-12-01
719            Outside Sales   83995.40 2022-12-01
791             Retail Sales  100521.87 2022-12-01
863     Sales Administration   24037.29 2022-12-01
935          Sales Analytics  110008.72 2022-12-01
1007        Sales Enablement   30265.16 2022-12-01
1079       Sales Engineering   25524.96 2022-12-01
1151        Sales Operations   94512.04 2022-12-01
1223           Sales Support   93343.62 2022-12-01
1295          Sales Training  116266.57 2022-12-01
1367  Strategic Partnerships   

Generating Jobdesk

In [9]:
df = pd.read_csv("Dataset.csv")

In [10]:
dep_name ={
  "departments": [
    {"Department": "Account Management", "KPi Achieved": 0.470105},
    {"Department": "Business Development", "KPi Achieved": 0.461890},
    {"Department": "Channel Sales", "KPi Achieved": 0.481059},
    {"Department": "Customer Success", "KPi Achieved": 0.484254},
    {"Department": "E-commerce Sales", "KPi Achieved": 0.456413},
    {"Department": "Enterprise Sales", "KPi Achieved": 0.472387},
    {"Department": "Inside Sales", "KPi Achieved": 0.486536},
    {"Department": "International Sales", "KPi Achieved": 0.456869},
    {"Department": "Key Accounts", "KPi Achieved": 0.486992},
    {"Department": "Outside Sales", "KPi Achieved": 0.459151},
    {"Department": "Retail Sales", "KPi Achieved": 0.452761},
    {"Department": "Sales Administration", "KPi Achieved": 0.467823},
    {"Department": "Sales Analytics", "KPi Achieved": 0.482885},
    {"Department": "Sales Enablement", "KPi Achieved": 0.488361},
    {"Department": "Sales Engineering", "KPi Achieved": 0.465084},
    {"Department": "Sales Operations", "KPi Achieved": 0.486079},
    {"Department": "Sales Support", "KPi Achieved": 0.479690},
    {"Department": "Sales Training", "KPi Achieved": 0.472843},
    {"Department": "Strategic Partnerships", "KPi Achieved": 0.476495}
  ]
}


In [11]:
dep_name_2 = dep_name['departments']
input_data = str(dep_name_2)


In [12]:
def ideal_organization(input_data, creds = creds, project_id = project_id):
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 300,
            "min_new_tokens": 1,
            "repetition_penalty": 1,
            "Temperature" : 0.7
            # "stop_sequences": [""]
        }
        llama2 = "meta-llama/llama-2-70b-chat"
        model = Model(
                model_id=llama2,
                params=parameters,
                credentials=creds,
                project_id=project_id)
        
        template = "<[INST] «SYS»\n"\
                "while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\n"\
                "Please ensure that your responses are socially unbiased and positive in nature.\n"\
                "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n"\
                "If you don't know the answer to a question, please don't share false information.\n"\
                "«/SYS»\n"\
                f"{input_data} is the KPI achieved ratio of a company's sales department\n"\
                "Choose one department that the kpi achieved ratio under 0.46\n"\
                "after chooese the department please give us the job description and thing to improve"\
                "[/INST]\n"\
                "Output:"

        # Generate text using the model
        generated_response = model.generate_text(prompt=template)

    # Return the results directly as JSON
        return generated_response


In [13]:
result = ideal_organization(input_data)
print(result)



Based on the given data, the department with the lowest KPI achieved ratio is "E-commerce Sales" with a ratio of 0.456413.

Job Description:
The E-commerce Sales department is responsible for managing the company's online sales channels, including its website, social media, and other e-commerce platforms. The team is tasked with driving sales growth through digital channels, providing excellent customer service, and ensuring a seamless online shopping experience for customers.

Things to Improve:

1. Digital Marketing Strategy: Review and optimize the department's digital marketing strategy to increase online visibility, drive more traffic to the website, and convert visitors into customers.
2. Customer Experience: Improve the overall customer experience by streamlining the online shopping process, providing clear product information, and offering multiple payment options.
3. Product Offerings: Analyze the product offerings and ensure they are competitive, relevant, and appealing to 

Generating most ideal organization Structure.

In [14]:
job = {
  "departments": [
    {
      "Department": "Account Management",
      "Responsibilities": [
        "Develop and maintain strong relationships with clients.",
        "Collaborate with clients to understand their needs and goals.",
        "Provide strategic guidance and solutions to ensure client success.",
        "Monitor and evaluate performance in account management."
      ]
    },
    {
      "Department": "Business Development",
      "Responsibilities": [
        "Identify and pursue new business opportunities and partnerships.",
        "Conduct market research and analysis to identify growth areas.",
        "Develop and implement strategies to drive business growth.",
        "Track and analyze performance in business development."
      ]
    },
    {
      "Department": "Channel Sales",
      "Responsibilities": [
        "Cultivate relationships with channel partners and distributors.",
        "Collaborate with partners to drive sales and meet targets.",
        "Provide support and training to channel partners.",
        "Monitor and assess performance in channel sales."
      ]
    },
    {
      "Department": "Customer Success",
      "Responsibilities": [
        "Onboard new customers and ensure a smooth transition.",
        "Proactively engage with customers to understand their needs.",
        "Address customer concerns and provide timely solutions.",
        "Measure and enhance customer satisfaction."
      ]
    },
    {
      "Department": "Enterprise Sales",
      "Responsibilities": [
        "Identify and target enterprise-level clients for sales opportunities.",
        "Lead complex sales processes and negotiations.",
        "Develop customized solutions to meet enterprise client needs.",
        "Assess and improve performance in enterprise sales."
      ]
    },
    {
      "Department": "Inside Sales",
      "Responsibilities": [
        "Conduct outbound and inbound sales activities.",
        "Collaborate with other departments to support sales efforts.",
        "Utilize sales tools and technology for effective communication.",
        "Evaluate and optimize performance in inside sales."
      ]
    },
    {
      "Department": "International Sales",
      "Responsibilities": [
        "Develop and execute strategies to expand international sales.",
        "Navigate and comply with international business regulations.",
        "Cultivate relationships with clients in different regions.",
        "Assess and enhance performance in international sales."
      ]
    },
    {
      "Department": "Key Accounts",
      "Responsibilities": [
        "Focus on maintaining and growing key client accounts.",
        "Conduct regular account reviews and strategy sessions.",
        "Collaborate with cross-functional teams to meet client needs.",
        "Evaluate and improve performance related to key accounts."
      ]
    },
    {
      "Department": "Outside Sales",
      "Responsibilities": [
        "Conduct face-to-face meetings with potential clients.",
        "Develop and maintain a network of contacts for business development.",
        "Represent the company at industry events and conferences.",
        "Assess and optimize performance in outside sales."
      ]
    },
    {
      "Department": "Retail Sales",
      "Responsibilities": [
        "Drive sales through retail channels.",
        "Establish and maintain relationships with retail partners.",
        "Implement strategies to increase product visibility and sales.",
        "Monitor and assess performance in retail sales."
      ]
    },
    {
      "Department": "Sales Administration",
      "Responsibilities": [
        "Provide administrative support to the sales team.",
        "Assist in the coordination of sales activities and events.",
        "Manage documentation and data related to sales processes.",
        "Evaluate and improve performance in sales administration."
      ]
    },
    {
      "Department": "Sales Analytics",
      "Responsibilities": [
        "Utilize analytics for data-driven insights in sales strategies.",
        "Conduct regular analysis of sales data and trends.",
        "Provide insights to support strategic decision-making in sales.",
        "Monitor and assess performance in sales analytics."
      ]
    },
    {
      "Department": "Sales Enablement",
      "Responsibilities": [
        "Enable the sales team with tools and resources for success.",
        "Develop and deliver training programs for sales effectiveness.",
        "Implement processes to streamline sales operations.",
        "Evaluate and optimize performance in sales enablement."
      ]
    },
    {
      "Department": "Sales Engineering",
      "Responsibilities": [
        "Combine technical expertise with sales strategies.",
        "Collaborate with the sales team to understand technical requirements.",
        "Provide technical support in sales presentations and discussions.",
        "Assess and enhance performance in sales engineering."
      ]
    },
    {
      "Department": "Sales Operations",
      "Responsibilities": [
        "Oversee and optimize sales processes and operations.",
        "Collaborate with various teams to improve sales efficiency.",
        "Implement tools and systems to enhance sales operations.",
        "Evaluate and optimize performance in sales operations."
      ]
    },
    {
      "Department": "Sales Support",
      "Responsibilities": [
        "Provide support to the sales team for smoother operations.",
        "Assist in managing customer inquiries and requests.",
        "Collaborate with internal teams to address sales support needs.",
        "Evaluate and improve performance in sales support."
      ]
    },
    {
      "Department": "Sales Training",
      "Responsibilities": [
        "Conduct training programs for the sales team.",
        "Develop and deliver content for sales training sessions.",
        "Assess and address the training needs of the sales team.",
        "Evaluate and optimize performance in sales training."
      ]
    },
    {
      "Department": "Strategic Partnerships",
      "Responsibilities": [
        "Develop and maintain strategic partnerships for business growth.",
        "Identify potential partners and opportunities for collaboration.",
        "Negotiate and establish mutually beneficial partnerships.",
        "Assess and enhance performance in strategic partnerships."
      ]
    }
  ]
}


In [15]:
merge_dept = {
    "merging": {
        "Department": "E-commerce Sales",
        "Responsibilities": [
            "Manage and optimize product listings on e-commerce platforms.",
            "Implement digital marketing strategies to drive online sales.",
            "Monitor and analyze e-commerce metrics to improve performance.",
            "Evaluate and enhance performance in e-commerce sales."
        ]
    }
}

In [16]:
def ideal_structure(dept_job,merge, creds = creds, project_id = project_id):
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 300,
            "min_new_tokens": 1,
            "repetition_penalty": 1,
            "Temperature" : 0.7
            # "stop_sequences": [""]
        }
        llama2 = "meta-llama/llama-2-70b-chat"
        model = Model(
                model_id=llama2,
                params=parameters,
                credentials=creds,
                project_id=project_id)
        
        template = "<[INST] «SYS»\n"\
                "while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\n"\
                "Please ensure that your responses are socially unbiased and positive in nature.\n"\
                "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n"\
                "If you don't know the answer to a question, please don't share false information.\n"\
                "«/SYS»\n"\
                f"based on the jobdesk could you give us insight is it possible for {merge_dept} merge to the department in {job}\n"\
                "[/INST]\n"\
                "Output:"

        # Generate text using the model
        generated_response = model.generate_text(prompt=template)

    # Return the results directly as JSON
        return generated_response


In [17]:
result = ideal_structure(job,merge_dept)
print(result)



It is possible to merge the 'E-commerce Sales' department with other departments in the sales organization. Here are some suggestions:

1. Merge with 'Account Management': As e-commerce sales involve managing online storefronts and customer relationships, merging with account management can help create a more comprehensive customer experience. The combined department can focus on building strong relationships with customers, managing their online presence, and driving sales through various channels.
2. Merge with 'Business Development': E-commerce sales can be an essential part of business development, especially when expanding into new markets or launching new products. By merging with business development, the e-commerce sales team can leverage their expertise in digital marketing and online sales to identify new opportunities and drive growth.
3. Merge with 'Channel Sales': Channel sales involve managing relationships with distributors, wholesalers, and retailers. By merging with 

Revenue Growth from the last revenue.

In [27]:
pred_data = {
  "revenue_prediction": [
    {
      "Retail Sales": "102784.99",
      "Sales Administration": "24678.92",
      "Account Management": "52845.89",
      "Business Development": "54250.89",
      "Channel Sales": "25000.00",
      "Customer Success": "55642.89",
      "E-commerce Sales": "75000",
      "Enterprise Sales": "59878.92",
      "Inside Sales": "72342.89",
      "International Sales": "58924.87",
      "Key Accounts": "78234.52",
      "Outside Sales": "68421.32",
      "Retail Sales": "69544.89",
      "Sales Administration": "24678.92",
      "Sales Analytics": "78943.89",
      "Sales Enablement": "23542.15",
      "Sales Engineering": "24567.89",
      "Sales Operations": "65425.89",
      "Sales Support": "98745.26",
      "Sales Training": "89421.25",
      "Strategic Partnerships": "85000"
    }
  ]
}


In [28]:
df_only_2022_11 = df_new[df_new["YearMonth"] == "2022-11"]
df_only_2022_11.tail(20)



,Department,Revenue,YearMonth
70,Account Management,64799.64,2022-11-01
142,Business Development,61950.15,2022-11-01
214,Channel Sales,27661.48,2022-11-01
286,Customer Success,58988.97,2022-11-01
358,E-commerce Sales,77430.95,2022-11-01
430,Enterprise Sales,57580.44,2022-11-01
502,Inside Sales,84916.04,2022-11-01
574,International Sales,53415.52,2022-11-01
646,Key Accounts,75239.96,2022-11-01
718,Outside Sales,67310.19,2022-11-01


In [29]:
last_revenue = {
  "real_revenue": {
    "Account Management": 64799.64,
    "Business Development": 61950.15,
    "Channel Sales": 27661.48,
    "Customer Success": 58988.97,
    "E-commerce Sales": 77430.95,
    "Enterprise Sales": 57580.44,
    "Inside Sales": 84916.04,
    "International Sales": 53415.52,
    "Key Accounts": 75239.96,
    "Outside Sales": 67310.19,
    "Retail Sales": 67396.28,
    "Sales Administration": 21590.11,
    "Sales Analytics": 74278.92,
    "Sales Enablement": 21327.32,
    "Sales Engineering": 23452.96,
    "Sales Operations": 63181.58,
    "Sales Support": 97633.48,
    "Sales Training": 87132.42,
    "Strategic Partnerships": 82990.05
  }
}



In [48]:
def rev_growth(real,pred, creds = creds, project_id = project_id):
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 1000,
            "min_new_tokens": 1,
            "repetition_penalty": 1,
            #"Temperature" : 0.7
            # "stop_sequences": [""]
        }
        llama2 = "meta-llama/llama-2-70b-chat"
        model = Model(
                model_id=llama2,
                params=parameters,
                credentials=creds,
                project_id=project_id)
        
        template = "<[INST] «SYS»\n"\
                "while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\n"\
                "Please ensure that your responses are socially unbiased and positive in nature.\n"\
                "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.\n"\
                "If you don't know the answer to a question, please don't share false information.\n"\
                "«/SYS»\n"\
                f"{last_revenue} is the last revenue data of a company.\n"\
                f"{pred_data} is a prediction of a company revenue for the coming month. \n"\
                f"Calculate the revenue growth from {last_revenue} with {pred_data}.\n"\
                "[/INST]\n"\
                "Output:"

        # Generate text using the model
        generated_response = model.generate_text(prompt=template)

    # Return the results directly as JSON
        return generated_response


In [49]:
rev_growth_result = rev_growth(last_revenue,pred_data)
print(rev_growth_result)



The revenue growth from the current revenue data to the predicted revenue data is:

* Retail Sales: 69544.89 - 67396.28 = 2148.61 (3.19%)
* Sales Administration: 24678.92 - 21590.11 = 3088.81 (14.3%)
* Account Management: 52845.89 - 64799.64 = -11953.75 (-18.5%)
* Business Development: 54250.89 - 61950.15 = -7700.06 (-12.5%)
* Channel Sales: 25000.00 - 27661.48 = -2661.48 (-9.6%)
* Customer Success: 55642.89 - 58988.97 = -3346.08 (-5.7%)
* E-commerce Sales: 75000 - 77430.95 = -2430.95 (-3.2%)
* Enterprise Sales: 59878.92 - 57580.44 = 2298.48 (4.0%)
* Inside Sales: 72342.89 - 84916.04 = -12573.15 (-14.8%)
* International Sales: 58924.87 - 53415.52 = 5509.35 (10.3%)
* Key Accounts: 78234.52 - 75239.96 = 2994.56 (4.0%)
* Outside Sales: 68421.32 - 67310.19 = 1111.13 (1.7%)
* Sales Analytics: 78943.89 - 74278.92 = 4664.97 (6.3%)
* Sales Enablement: 23542.15 - 21327.32 = 2214.83 (10.4%)
* Sales Engineering: 24567.89 - 23452.96 = 1114.93 (4.8%)
* Sales Operations: 65425.89 - 63181.58 = 2244